# Scraping WikiArt by Selenium

### Author: Yangyu Wang
### Date: Jan 18, 2025

## Generate artist names from existing dataset

In [1]:
import pandas as pd
import csv
import unicodedata

def remove_accents(input_str):
    """
    Convert Spanish and other accented characters to their English equivalent.
    """
    return unicodedata.normalize('NFKD', input_str).encode('ASCII', 'ignore').decode('utf-8')

file_path = "wikiart_expanded/wikiart_expanded.csv"
data = pd.read_csv(file_path)

unique_artists = data['Artist'].dropna().unique()

print(unique_artists[:10])

['Ancient Egypt' 'Ancient Greek Painting and Sculpture'
 'Ancient Greek Pottery' 'Apelles' 'Fayum portrait' 'Cricorps'
 'Teddy Cobeña' 'Thiago Boecan' 'Edward Burne-Jones' 'Orthodox Icons']


In [46]:
unique_artists[:5]

array(['Ancient Egypt', 'Ancient Greek Painting and Sculpture',
       'Ancient Greek Pottery', 'Apelles', 'Fayum portrait'], dtype=object)

## Open firefox

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Firefox()

The geckodriver version (0.34.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (134.0.1); currently, geckodriver 0.35.0 is recommended for firefox 134.*, so it is advised to delete the driver in PATH and retry


## Functions to extract artist information and artworks contents

In [41]:
def extract_artist_info(artist_name, url_name = None):

    if url_name:
        artist = url_name
    else:   
        artist = remove_accents(artist_name).lower().replace('.', '').replace(' ', '-')

    artist_info = {"name": artist_name, "url_name": artist}

    url_artist = f"https://www.wikiart.org/en/{artist}"
    #url_artworks = f"https://www.wikiart.org/en/{artist}/all-works/text-list"

    driver.get(url_artist)
    
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.TAG_NAME, "h3"))
        )
    except:
        print("Error (404):", artist_name)
        return artist_name

    try:
        birth_date_element = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="birthDate"]')
        artist_info["Birth Date"] = birth_date_element.text.strip()
    except:
        artist_info["Birth Date"] = None

    try:
        death_date_element = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="deathDate"]')
        artist_info["Death Date"] = death_date_element.text.strip()
    except:
        artist_info["Death Date"] = None

    try:
        birth_place_element = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="birthPlace"]')
        artist_info["Birth place"] = birth_place_element.text.strip()
    except:
        artist_info["Birth place"] = None

    try:
        death_place_element = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="deathPlace"]')
        artist_info["Death place"] = death_place_element.text.strip()
    except:
        artist_info["Death place"] = None

    li_elements = driver.find_elements(By.TAG_NAME, "li")

    for element in li_elements:
        try:
            field_name = element.find_element(By.TAG_NAME, "s").text.strip().replace(":", "")
        except:
            continue
            
        try:
            value_elements = element.find_elements(By.XPATH, ".//span/a | .//a")
            value_texts = [v.text.strip() for v in value_elements if v.text.strip()]

            if value_texts:
                artist_info[field_name] = "|".join(value_texts)
        except:
            artist_info[field_name] = None

    try:
        wiki_element = driver.find_element(By.CSS_SELECTOR, ".truncated-link a.truncate.external")
        artist_info["Wikipedia"] = wiki_element.get_attribute("href")
    except:
        artist_info["Wikipedia"] = None

    return(artist_info)


In [29]:
def extract_artist_work(artist_name, url_name = None):

    if url_name:
        artist = url_name
    else:   
        artist = remove_accents(artist_name).lower().replace('.', '').replace(' ', '-')
    
    artworks = []

    url_artworks = f"https://www.wikiart.org/en/{artist}/all-works/text-list"

    driver.get(url_artworks)

    painting_elements = driver.find_elements(By.CSS_SELECTOR, ".painting-list-text-row")

    for element in painting_elements:
        try:
            artwork_element = element.find_element(By.TAG_NAME, "a")
            artwork_name = artwork_element.text.strip()
            artwork_link = artwork_element.get_attribute("href")

            try:
                year_element = element.find_element(By.TAG_NAME, "span")
                artwork_year = year_element.text.strip().replace(",", "")
            except:
                artwork_year = None

            # 存储数据
            artworks.append({
                "Artist_name": artist,
                "Artwork Name": artwork_name,
                "Year": artwork_year,
                "Link": artwork_link
            })
        except:
            continue

    return artworks

## Extract artist information and artworks contents

In [5]:
from tqdm import tqdm 
import time
import random


In [ ]:

artist_data = []
not_exist = []
artworks_data = []

for artist in tqdm(unique_artists, desc="Processing Artists", unit="artist"):
    artist_info = extract_artist_info(artist)
    if isinstance(artist_info, dict):
        artist_data.append(artist_info)
        time.sleep(random.randint(3, 5))
        artworks_data = artworks_data + extract_artist_work(artist)
        time.sleep(random.randint(1, 3))
    else:
        not_exist.append(artist_info)


Processing Artists:   2%|▏         | 50/2174 [08:57<5:53:57, 10.00s/artist]

Error (404): Paweł Kluza


Processing Artists:   6%|▌         | 122/2174 [20:54<5:32:42,  9.73s/artist]

Error (404): [ a y s h ]


Processing Artists:   6%|▌         | 134/2174 [22:55<6:31:44, 11.52s/artist]

...

Processing Artists:  94%|█████████▍| 2050/2174 [5:30:28<19:48,  9.58s/artist]

Error (404): Luis Álvarez Roure


Processing Artists:  96%|█████████▌| 2083/2174 [5:35:38<13:40,  9.02s/artist]

Error (404): Shin Yoon-bok


Processing Artists:  98%|█████████▊| 2130/2174 [5:43:30<07:24, 10.11s/artist]

## Save into csv files

In [11]:
def extract_all_keys(dict_list):
    """
    Extract all unique keys from a list of dictionaries.
    Ensures that different dictionaries with different keys are fully accounted for.
    """
    ordered_keys = []
    seen_keys = set()
    
    for dictionary in dict_list:
        for key in dictionary.keys():
            if key not in seen_keys:
                ordered_keys.append(key)
                seen_keys.add(key)
                
    return ordered_keys

In [52]:
all_keys_artist = extract_all_keys(artist_data)

with open("artist_data/artist_data_new.csv", mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=all_keys_artist)

    writer.writeheader()

    for artist in tqdm(artist_data, desc="Saving artist info", unit="artist"):
        writer.writerow({key: artist.get(key, "") for key in all_keys_artist})

Saving artist info: 100%|██████████| 878/878 [00:00<00:00, 30261.64artist/s]


In [ ]:
all_keys_artworks = extract_all_keys(artworks_data)

with open("artist_data/artist_artwork.csv", mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=all_keys_artworks)

    writer.writeheader()

    for artist in tqdm(artworks_data, desc="Saving artwork info", unit="artwork"):
        writer.writerow({key: artist.get(key, "") for key in all_keys_artworks})

Saving artwork info: 100%|██████████| 80693/80693 [00:00<00:00, 91055.27artwork/s]


## Re-scraping for not found items

In [ ]:
with open("to_search.csv", mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["name", "url"])

    writer.writeheader()

    for artist in not_exist:
        writer.writerow({"name": artist})


Code the urls by human

In [42]:
with open("to_search_completed.csv", mode='r', encoding='utf-8', newline='') as file:
    reader = csv.reader(file)
    lines = [(row[0], row[1]) for row in reader][1:]


artist_data_notfound = []
not_exist_notfound = []
artworks_data_notfound = []

for artist, url in tqdm(lines, desc="Processing Artists", unit="artist"):
    artist_info = extract_artist_info(artist, url)
    if isinstance(artist_info, dict):
        artist_data_notfound.append(artist_info)
        time.sleep(1)
        artworks_data_notfound = artworks_data_notfound + extract_artist_work(artist, url)
        time.sleep(1)
    else:
        not_exist_notfound.append(artist_info)

Processing Artists: 100%|██████████| 4/4 [00:18<00:00,  4.66s/artist]


In [44]:
all_keys_artist = extract_all_keys(artist_data_notfound)

with open("artist_data/artist_data_notfound.csv", mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=all_keys_artist)

    writer.writeheader()

    for artist in tqdm(artist_data_notfound, desc="Saving artist info", unit="artist"):
        writer.writerow({key: artist.get(key, "") for key in all_keys_artist})

#deleted after merging

Saving artist info: 100%|██████████| 4/4 [00:00<00:00, 10149.56artist/s]


In [43]:
all_keys_artworks = extract_all_keys(artworks_data_notfound)

with open("artist_data/artist_artwork_notfound.csv", mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=all_keys_artworks)

    writer.writeheader()

    for artist in tqdm(artworks_data_notfound, desc="Saving artwork info", unit="artwork"):
        writer.writerow({key: artist.get(key, "") for key in all_keys_artworks})

#deleted after merging

Saving artwork info: 100%|██████████| 40/40 [00:00<00:00, 56660.64artwork/s]
